In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset_preprocessing import fetch_preprocessed_data

In [3]:
# fetch_preprocessed_data()

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
from dataset_preprocessing import TokenInfo

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [7]:
from tqdm import tqdm
import random

In [8]:
token_info = TokenInfo()

...Loading dataset...
...Loading complete...


In [9]:
model_id = "microsoft/phi-1_5"
model_revision = "349cf8b5e81fd5f791d1740da5de1313a0419bbd" # latest as of feb 1st

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
vocab = tokenizer.get_vocab()
len(vocab)

50295

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision=model_revision,
    trust_remote_code=True,
    # be careful with this?
    torch_dtype=torch.float16,
    # attn_implementation="flash_attention_2",
)

In [12]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2048,), e

In [13]:
token_samples = token_info.top_n(10_000) # approx top 20% tokens

In [14]:
from importances import compute_delta_loss_importances

In [15]:
importance_dict = {}
# take longer sequences
for i, t_info in tqdm(enumerate(token_samples)):
    examples = token_info.get_prefixes(t_info[0], 10, 10)
    examples = torch.tensor(examples, device="cuda:0")
    importances, sample_inputs = compute_delta_loss_importances(model, examples)
    importance_dict[t_info] = (importances, sample_inputs)
    if (i+1) % 100 == 0:
        pd.to_pickle(importance_dict, f"importances_inputs/importance_{i}.pkl")        
        del importance_dict
        importance_dict = {}


10000it [2:54:58,  1.05s/it]


In [19]:
importance_dict = pd.read_pickle("importances_inputs/importance_99.pkl")

In [21]:
keys = list(importance_dict.keys())
inputs_0 = importance_dict[keys[0]]

In [22]:
keys

[(13, '.', 3932482),
 (11, ',', 3259473),
 (262, ' the', 3155352),
 (290, ' and', 2155455),
 (286, ' of', 1956767),
 (284, ' to', 1825320),
 (198, '\n', 1706000),
 (257, ' a', 1478621),
 (287, ' in', 1271252),
 (12, '-', 926863),
 (318, ' is', 840309),
 (329, ' for', 776238),
 (492, '..', 613565),
 (351, ' with', 592647),
 (447, '�', 561724),
 (326, ' that', 550820),
 (319, ' on', 495997),
 (25, ':', 493786),
 (357, ' (', 405048),
 (247, '�', 393665),
 (389, ' are', 386448),
 (383, ' The', 381717),
 (345, ' you', 363524),
 (355, ' as', 362606),
 (314, ' I', 344230),
 (307, ' be', 340175),
 (82, 's', 336810),
 (340, ' it', 323477),
 (416, ' by', 310280),
 (422, ' from', 304457),
 (379, ' at', 282352),
 (8, ')', 278764),
 (281, ' an', 273679),
 (373, ' was', 269707),
 (423, ' have', 267411),
 (393, ' or', 264426),
 (428, ' this', 257714),
 (534, ' your', 240872),
 (481, ' will', 232736),
 (460, ' can', 229507),
 (468, ' has', 220713),
 (532, ' -', 207396),
 (338, "'s", 191937),
 (14, '/'

In [23]:
list(inputs_0.keys())

AttributeError: 'tuple' object has no attribute 'keys'

In [ ]:
inputs_0[0]

In [ ]:
inputs_0[0].size()

In [ ]:
examples.size()